In [1]:
import numpy as np
import pandas as pd

In [2]:
order = pd.read_parquet('order/000010.parquet')  # 逐笔委托
trade = pd.read_parquet('trade/000010.parquet')  # 逐笔成交
shot = pd.read_parquet('level/000010.parquet')  # 快照档位
snap = pd.read_parquet('hq_snap_spot/000010.parquet')  # 快照

In [3]:
order.drop(['tradedate', 'OrigTime', 'SendTime', 'recvtime', 'dbtime', 'ChannelNo', 
'MDStreamID', 'SecurityID', 'SecurityIDSource', 
'Contactor', 'ContactInfo', 'ExpirationDays', 'ExpirationType', 'ConfirmID'], axis = 1, inplace=True)
trade.drop(['tradedate', 'OrigTime', 'SendTime', 'recvtime', 'dbtime', 'ChannelNo', 
'MDStreamID', 'SecurityID', 'SecurityIDSource'], axis = 1, inplace = True)
shot.drop(['TradeDate', 'SendTime', 'RecvTime', 'DBTime', 'ChannelID', 'SecurityID', 'SecurityIDSource', 'MDStreamID', 
'NumOrders_B1', 'NoOrders_B1', 'OrderQty_B1', 'NumOrders_S1', 'NoOrders_S1', 'OrderQty_S1'], axis = 1, inplace = True)
snap.drop(['TradeDate', 'SendTime', 'recvtime', 'dbtime', 'ChannelNo', 
'SecurityID', 'SecurityIDSource', 'MDStreamID', 'PreNAV', 'RealTimeNAV', 'WarrentPremiumRate', 
'TotalLongPosition', 'c32', 'c33', 'c34'], axis = 1, inplace = True)

In [4]:
# combine snap and shot (snapNshot)

snapNshot = pd.merge(snap, shot, on='OrigTime')
snapNshot.columns

Index(['OrigTime', 'PreClosePx', 'PxChange1', 'PxChange2', 'OpenPx', 'HighPx',
       'LowPx', 'LastPx', 'NumTrades', 'TotalVolumeTrade', 'TotalValueTrade',
       'PERatio1', 'PERatio2', 'TradingPhaseCode', 'totalofferqty',
       'weightedavgofferpx', 'totalbidqty', 'weightedavgbidpx', 'UpLimitPx',
       'DownLimitPx', 'OfferPX1', 'BidPX1', 'OfferSize1', 'BidSize1',
       'OfferPX2', 'BidPX2', 'OfferSize2', 'BidSize2', 'OfferPX3', 'BidPX3',
       'OfferSize3', 'BidSize3', 'OfferPX4', 'BidPX4', 'OfferSize4',
       'BidSize4', 'OfferPX5', 'BidPX5', 'OfferSize5', 'BidSize5', 'OfferPX6',
       'BidPX6', 'OfferSize6', 'BidSize6', 'OfferPX7', 'BidPX7', 'OfferSize7',
       'BidSize7', 'OfferPX8', 'BidPX8', 'OfferSize8', 'BidSize8', 'OfferPX9',
       'BidPX9', 'OfferSize9', 'BidSize9', 'OfferPX10', 'BidPX10',
       'OfferSize10', 'BidSize10'],
      dtype='object')

In [5]:
# combine order and trade (orderNtrade)

origtime = snapNshot.OrigTime.values  # 快照生成时间节点
orderNtrade =  pd.concat([order.rename({'OrderQty':'Qty', 'TransactTime':'Time', 'OrderType':'Type'}, axis=1), 
trade.rename({'TradeQty':'Qty', 'tradetime':'Time', 'ExecType':'Type'}, axis=1), 
pd.DataFrame({'Time': origtime, 'Type': ['s' for _ in range(len(origtime))]})], ignore_index=True).sort_values(by=['Time', 'ApplSeqNum']).reset_index(drop = True)
orderNtrade.columns

Index(['ApplSeqNum', 'Price', 'Qty', 'Time', 'Side', 'Type', 'BidApplSeqNum',
       'OfferApplSeqNum'],
      dtype='object')

In [6]:
# recreate snapNshot from order and trade

PreClosePx = 3.91
PxChange1 = np.nan
PxChange2 = np.nan
OpenPx = np.nan
HighPx = np.nan
LowPx = np.nan
LastPx = np.nan
NumTrades = 0
TotalVolumeTrade = 0
TotalValueTrade = 0
PERatio1 = np.nan
PERatio2 = np.nan
TradingPhaseCode = np.nan
totalofferqty = np.nan
weightedavgofferpx = np.nan
totalbidqty = np.nan
weightedavgbidpx = np.nan
UpLimitPx = np.around(PreClosePx * 1.1, decimals=2)
DownLimitPx = np.around(PreClosePx * 0.9, decimals=2)

recreated = pd.DataFrame(columns = snapNshot.columns).astype({'OrigTime':'int64'})

df_bid = pd.DataFrame(columns=['price', 'qty', 'appl'])  # bid order book
df_offer = pd.DataFrame(columns=['price', 'qty', 'appl'])  # offer order book

for index, row in orderNtrade.iterrows():

    _appl, _price, _qty, _time, _side, _type, _bidappl, _offerappl = row.values

    if _type == '2':  # 下限价单

        if _side == 1:  # 买
            df_bid = pd.concat([df_bid, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
            df_bid.sort_values(by=['price', 'appl'], ascending=[False,True], inplace=True)

        if _side == 2:  # 卖
            df_offer = pd.concat([df_offer, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
            df_offer.sort_values(by=['price', 'appl'], ascending=[True,True], inplace=True)

    if _type == '1':  # 下市价单

        if _side == 1:  # 买
            _price = df_offer.iloc[0,0]
            df_bid = pd.concat([df_bid, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
            df_bid.sort_values(by=['price', 'appl'], ascending=[False,True], inplace=True)

        if _side == 2:  # 卖
            _price = df_bid.iloc[0,0]
            df_offer = pd.concat([df_offer, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
            df_offer.sort_values(by=['price', 'appl'], ascending=[True,True], inplace=True)

    if _type == 'U':  # 下本方最优单

        if _side == 1:  # 买
            _price = df_bid.iloc[0,0]
            df_bid = pd.concat([df_bid, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
            df_bid.sort_values(by=['price', 'appl'], ascending=[False,True], inplace=True)
                
        if _side == 2:  # 卖
            _price = df_offer.iloc[0,0]
            df_offer = pd.concat([df_offer, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
            df_offer.sort_values(by=['price', 'appl'], ascending=[True,True], inplace=True)

    if _type == '4':  # 撤单

        if _bidappl > 0:
            df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] = df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] - _qty
            df_bid = df_bid[df_bid['qty'] > 0]
        if _offerappl > 0:
            df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] = df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] - _qty
            df_offer = df_offer[df_offer['qty'] > 0]

    if _type == 'F':  # 成交

        if np.isnan(OpenPx):
            OpenPx = _price
        if np.isnan(HighPx) or _price > HighPx:
            HighPx = _price
        if np.isnan(LowPx) or _price < LowPx:
            LowPx = _price
        LastPx = _price
        PxChange1 = (LastPx - PreClosePx)/PreClosePx
        PxChange2 = (LastPx - OpenPx)/OpenPx
        NumTrades = NumTrades + 1
        TotalVolumeTrade = TotalVolumeTrade + _qty
        TotalValueTrade = TotalValueTrade + _price * _qty

        df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] = df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] - _qty
        df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] = df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] - _qty

        df_bid = df_bid[df_bid['qty'] > 0]
        df_offer = df_offer[df_offer['qty'] > 0]

    if _type == 's':  # 快照节点

        totalbidqty = df_bid['qty'].sum()
        totalofferqty = df_offer['qty'].sum()
        if len(df_bid.price) > 0:
            weightedavgbidpx = np.dot(df_bid.price, df_bid.qty)/totalbidqty
        if len(df_offer.price) > 0:
            weightedavgofferpx = np.dot(df_offer.price, df_offer.qty)/totalofferqty

        if len(df_bid.price.unique()) >= 10:
            BidPxs = df_bid.groupby('price', sort = False).sum().head(10).index.values
            BidQtys = df_bid.groupby('price', sort = False).sum().head(10).qty.values
        elif len(df_bid.price.unique()) > 0 and len(df_bid.price.unique()) < 10:
            BidPxs = df_bid.groupby('price', sort = False).sum().index.values
            BidPxs = np.append(BidPxs, [np.nan for _ in range(10-len(df_bid.price.unique()))])
            BidQtys = df_bid.groupby('price', sort = False).sum().qty.values
            BidQtys = np.append(BidQtys, [np.nan for _ in range(10-len(df_bid.price.unique()))])
        else:
            BidPxs = BidQtys = np.array([np.nan for _ in range(10)])

        if len(df_offer.price.unique()) >= 10:
            OfferPxs = df_offer.groupby('price', sort = False).sum().head(10).index.values
            OfferQtys = df_offer.groupby('price', sort = False).sum().head(10).qty.values
        elif len(df_offer.price.unique()) > 0 and len(df_offer.price.unique()) < 10:
            OfferPxs = df_offer.groupby('price', sort = False).sum().index.values
            OfferPxs = np.append(OfferPxs, [np.nan for _ in range(10-len(df_offer.price.unique()))])
            OfferQtys = df_offer.groupby('price', sort = False).sum().qty.values
            OfferQtys = np.append(OfferQtys, [np.nan for _ in range(10-len(df_offer.price.unique()))])
        else:
            OfferPxs = OfferQtys = np.array([np.nan for _ in range(10)])

        recreated = recreated.append(pd.Series([_time, PreClosePx, PxChange1, PxChange2, OpenPx, HighPx, LowPx, LastPx,
            NumTrades, TotalVolumeTrade, TotalValueTrade, PERatio1, PERatio2, TradingPhaseCode,
            totalofferqty, weightedavgofferpx, totalbidqty, weightedavgbidpx, UpLimitPx, DownLimitPx,
            OfferPxs[0], BidPxs[0], OfferQtys[0], BidQtys[0],
            OfferPxs[1], BidPxs[1], OfferQtys[1], BidQtys[1],
            OfferPxs[2], BidPxs[2], OfferQtys[2], BidQtys[2],
            OfferPxs[3], BidPxs[3], OfferQtys[3], BidQtys[3],
            OfferPxs[4], BidPxs[4], OfferQtys[4], BidQtys[4],
            OfferPxs[5], BidPxs[5], OfferQtys[5], BidQtys[5],
            OfferPxs[6], BidPxs[6], OfferQtys[6], BidQtys[6],
            OfferPxs[7], BidPxs[7], OfferQtys[7], BidQtys[7],
            OfferPxs[8], BidPxs[8], OfferQtys[8], BidQtys[8],
            OfferPxs[9], BidPxs[9], OfferQtys[9], BidQtys[9]], index = recreated.columns), ignore_index = True)

In [7]:
# sanity check

diff = recreated - snapNshot

# Function that takes in order, trade, shot, snap and outputs snapshot

In [8]:
def SZtick2snapshot(order, trade, origtime, PreClosePx):

    '''
    order : 逐笔委托数据
    trade : 逐笔成交数据
    origtime : 快照生成时间节点 (numpy array of int)
    PreClosePx : 昨日收盘价 (float)
    '''

    snapshot_columns = ['OrigTime', 'PreClosePx', 'PxChange1', 'PxChange2', 'OpenPx', 'HighPx',
       'LowPx', 'LastPx', 'NumTrades', 'TotalVolumeTrade', 'TotalValueTrade',
       'PERatio1', 'PERatio2', 'TradingPhaseCode', 'totalofferqty',
       'weightedavgofferpx', 'totalbidqty', 'weightedavgbidpx', 'UpLimitPx',
       'DownLimitPx', 'OfferPX1', 'BidPX1', 'OfferSize1', 'BidSize1',
       'OfferPX2', 'BidPX2', 'OfferSize2', 'BidSize2', 'OfferPX3', 'BidPX3',
       'OfferSize3', 'BidSize3', 'OfferPX4', 'BidPX4', 'OfferSize4',
       'BidSize4', 'OfferPX5', 'BidPX5', 'OfferSize5', 'BidSize5', 'OfferPX6',
       'BidPX6', 'OfferSize6', 'BidSize6', 'OfferPX7', 'BidPX7', 'OfferSize7',
       'BidSize7', 'OfferPX8', 'BidPX8', 'OfferSize8', 'BidSize8', 'OfferPX9',
       'BidPX9', 'OfferSize9', 'BidSize9', 'OfferPX10', 'BidPX10',
       'OfferSize10', 'BidSize10']

    order.drop(['tradedate', 'OrigTime', 'SendTime', 'recvtime', 'dbtime', 'ChannelNo', 
    'MDStreamID', 'SecurityID', 'SecurityIDSource', 
    'Contactor', 'ContactInfo', 'ExpirationDays', 'ExpirationType', 'ConfirmID'], axis = 1, inplace=True)
    trade.drop(['tradedate', 'OrigTime', 'SendTime', 'recvtime', 'dbtime', 'ChannelNo', 
    'MDStreamID', 'SecurityID', 'SecurityIDSource'], axis = 1, inplace = True)
    
    orderNtrade =  pd.concat([order.rename({'OrderQty':'Qty', 'TransactTime':'Time', 'OrderType':'Type'}, axis=1), 
        trade.rename({'TradeQty':'Qty', 'tradetime':'Time', 'ExecType':'Type'}, axis=1), 
        pd.DataFrame({'Time': origtime, 'Type': ['s' for _ in range(len(origtime))]})], ignore_index=True).sort_values(by=['Time', 'ApplSeqNum']).reset_index(drop = True)

    PxChange1 = np.nan
    PxChange2 = np.nan
    OpenPx = np.nan
    HighPx = np.nan
    LowPx = np.nan
    LastPx = np.nan
    NumTrades = 0
    TotalVolumeTrade = 0
    TotalValueTrade = 0
    PERatio1 = np.nan
    PERatio2 = np.nan
    TradingPhaseCode = np.nan
    totalofferqty = np.nan
    weightedavgofferpx = np.nan
    totalbidqty = np.nan
    weightedavgbidpx = np.nan
    UpLimitPx = np.around(PreClosePx * 1.1, decimals=2)
    DownLimitPx = np.around(PreClosePx * 0.9, decimals=2)

    recreated = pd.DataFrame(columns = snapshot_columns).astype({'OrigTime':'int64'})

    df_bid = pd.DataFrame(columns=['price', 'qty', 'appl'])  # bid order book
    df_offer = pd.DataFrame(columns=['price', 'qty', 'appl'])  # offer order book

    for index, row in orderNtrade.iterrows():

        _appl, _price, _qty, _time, _side, _type, _bidappl, _offerappl = row.values

        if _type == '2':  # 下限价单

            if _side == 1:  # 买
                df_bid = pd.concat([df_bid, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
                df_bid.sort_values(by=['price', 'appl'], ascending=[False,True], inplace=True)

            if _side == 2:  # 卖
                df_offer = pd.concat([df_offer, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
                df_offer.sort_values(by=['price', 'appl'], ascending=[True,True], inplace=True)

        if _type == '1':  # 下市价单

            if _side == 1:  # 买
                _price = df_offer.iloc[0,0]
                df_bid = pd.concat([df_bid, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
                df_bid.sort_values(by=['price', 'appl'], ascending=[False,True], inplace=True)

            if _side == 2:  # 卖
                _price = df_bid.iloc[0,0]
                df_offer = pd.concat([df_offer, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
                df_offer.sort_values(by=['price', 'appl'], ascending=[True,True], inplace=True)

        if _type == 'U':  # 下本方最优单

            if _side == 1:  # 买
                _price = df_bid.iloc[0,0]
                df_bid = pd.concat([df_bid, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
                df_bid.sort_values(by=['price', 'appl'], ascending=[False,True], inplace=True)
                    
            if _side == 2:  # 卖
                _price = df_offer.iloc[0,0]
                df_offer = pd.concat([df_offer, pd.DataFrame({'price':[_price],'qty':[_qty],'appl':[_appl]})])
                df_offer.sort_values(by=['price', 'appl'], ascending=[True,True], inplace=True)

        if _type == '4':  # 撤单

            if _bidappl > 0:
                df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] = df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] - _qty
                df_bid = df_bid[df_bid['qty'] > 0]
            if _offerappl > 0:
                df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] = df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] - _qty
                df_offer = df_offer[df_offer['qty'] > 0]

        if _type == 'F':  # 成交

            if np.isnan(OpenPx):
                OpenPx = _price
            if np.isnan(HighPx) or _price > HighPx:
                HighPx = _price
            if np.isnan(LowPx) or _price < LowPx:
                LowPx = _price
            LastPx = _price
            PxChange1 = (LastPx - PreClosePx)/PreClosePx
            PxChange2 = (LastPx - OpenPx)/OpenPx
            NumTrades = NumTrades + 1
            TotalVolumeTrade = TotalVolumeTrade + _qty
            TotalValueTrade = TotalValueTrade + _price * _qty

            df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] = df_bid.loc[df_bid['appl'] == _bidappl, 'qty'] - _qty
            df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] = df_offer.loc[df_offer['appl'] == _offerappl, 'qty'] - _qty

            df_bid = df_bid[df_bid['qty'] > 0]
            df_offer = df_offer[df_offer['qty'] > 0]

        if _type == 's':  # 快照节点

            totalbidqty = df_bid['qty'].sum()
            totalofferqty = df_offer['qty'].sum()
            if len(df_bid.price) > 0:
                weightedavgbidpx = np.dot(df_bid.price, df_bid.qty)/totalbidqty
            if len(df_offer.price) > 0:
                weightedavgofferpx = np.dot(df_offer.price, df_offer.qty)/totalofferqty

            if len(df_bid.price.unique()) >= 10:
                BidPxs = df_bid.groupby('price', sort = False).sum().head(10).index.values
                BidQtys = df_bid.groupby('price', sort = False).sum().head(10).qty.values
            elif len(df_bid.price.unique()) > 0 and len(df_bid.price.unique()) < 10:
                BidPxs = df_bid.groupby('price', sort = False).sum().index.values
                BidPxs = np.append(BidPxs, [np.nan for _ in range(10-len(df_bid.price.unique()))])
                BidQtys = df_bid.groupby('price', sort = False).sum().qty.values
                BidQtys = np.append(BidQtys, [np.nan for _ in range(10-len(df_bid.price.unique()))])
            else:
                BidPxs = BidQtys = np.array([np.nan for _ in range(10)])

            if len(df_offer.price.unique()) >= 10:
                OfferPxs = df_offer.groupby('price', sort = False).sum().head(10).index.values
                OfferQtys = df_offer.groupby('price', sort = False).sum().head(10).qty.values
            elif len(df_offer.price.unique()) > 0 and len(df_offer.price.unique()) < 10:
                OfferPxs = df_offer.groupby('price', sort = False).sum().index.values
                OfferPxs = np.append(OfferPxs, [np.nan for _ in range(10-len(df_offer.price.unique()))])
                OfferQtys = df_offer.groupby('price', sort = False).sum().qty.values
                OfferQtys = np.append(OfferQtys, [np.nan for _ in range(10-len(df_offer.price.unique()))])
            else:
                OfferPxs = OfferQtys = np.array([np.nan for _ in range(10)])

            recreated = recreated.append(pd.Series([_time, PreClosePx, PxChange1, PxChange2, OpenPx, HighPx, LowPx, LastPx,
                NumTrades, TotalVolumeTrade, TotalValueTrade, PERatio1, PERatio2, TradingPhaseCode,
                totalofferqty, weightedavgofferpx, totalbidqty, weightedavgbidpx, UpLimitPx, DownLimitPx,
                OfferPxs[0], BidPxs[0], OfferQtys[0], BidQtys[0],
                OfferPxs[1], BidPxs[1], OfferQtys[1], BidQtys[1],
                OfferPxs[2], BidPxs[2], OfferQtys[2], BidQtys[2],
                OfferPxs[3], BidPxs[3], OfferQtys[3], BidQtys[3],
                OfferPxs[4], BidPxs[4], OfferQtys[4], BidQtys[4],
                OfferPxs[5], BidPxs[5], OfferQtys[5], BidQtys[5],
                OfferPxs[6], BidPxs[6], OfferQtys[6], BidQtys[6],
                OfferPxs[7], BidPxs[7], OfferQtys[7], BidQtys[7],
                OfferPxs[8], BidPxs[8], OfferQtys[8], BidQtys[8],
                OfferPxs[9], BidPxs[9], OfferQtys[9], BidQtys[9]], index = recreated.columns), ignore_index = True)
    
    return recreated